In [35]:
import numpy as np
import scipy as sp
from scipy.integrate import quad_vec
import matplotlib.pyplot as plt
from manim import *
import ipywidgets

config.media_width = "75%"
config.verbosity = "WARNING"
#HYPERPARAMETERS
z_func = lambda t: (t-3)**2-4.5+3j*np.sin(2*np.pi*t/6)
T = 6
N = 3
dx = 0.01

## 1. Find Fourier Coefficients
$$
    c_{n} = \frac{1}{T} \int_{0}^{T} \boldsymbol{z} (t) e^{-2\pi ifnt} dt
$$

The Fourier coefficients represent all the frequency information of the T-periodic function $\boldsymbol{z}(t)$. The idea behind the coefficients is to assume that the periodic function can be broken down into sinusodal functions then find the amplitude of every T-periodic sinusiod that could potentially be included in the function. The magnitude of the nth coefficient represents the radius of the spinning vector with frequency f*n while the angle determines the starting angle for their period. 

In [36]:
#Integrals with complex numbers
def complex_quadrature_vec(func, a, b, **kwargs):
    def real_func(x):
        return np.real(func(x))
    def imag_func(x):
        return np.imag(func(x))
    real_integral = quad_vec(real_func, a, b, **kwargs)
    imag_integral = quad_vec(imag_func, a, b, **kwargs)
    return real_integral[0] + 1j*imag_integral[0]

#Compute integrals (in parralel)
def fourier_coefficients(z_func,T,N):
    n = np.arange(-N, N+1)
    func = lambda t: z_func(t)*np.exp(-2j*n*np.pi*t/T)
    return complex_quadrature_vec(func,0,T)/T


c = fourier_coefficients(z_func,T,N)
print(c)

[ 0.01823781-5.27355937e-16j  0.02251582-7.03141249e-16j
  0.02849658-1.43403807e-16j  0.03722003-2.63677968e-16j
  0.05066059-2.86807615e-16j  0.07295125-2.59052039e-16j
  0.11398633-1.11022302e-16j  0.20264237-5.18104078e-16j
  0.45594533-1.20274161e-16j  0.32378131-4.81096644e-16j
 -1.5       -7.40148683e-17j  3.32378131+8.88178420e-16j
  0.45594533+2.96059473e-16j  0.20264237+5.92118946e-16j
  0.11398633+2.22044605e-16j  0.07295125+1.85037171e-16j
  0.05066059-4.25585493e-16j  0.03722003-2.59052039e-16j
  0.02849658+1.11022302e-16j  0.02251582-4.53341068e-16j
  0.01823781+3.70074342e-16j]


## 2. Compute Fourier Series

$$ 
    z_{N} (t) = \sum_{n=-N}^{N} c_{n} e^{2\pi ifnt} 
$$

In [37]:
# use matrix multiplication to compute approximation through time in parallel
def compute_z_n(c,T,dx,N):
    t = np.linspace(0,T,int(T/dx+1))
    t = np.reshape(t, (len(t),1))

    n = np.arange(-N, N+1)
    n = np.reshape(n, (1,len(n)))

    t_n_matrix = np.matmul(t,n)
    exp_matrix = np.exp(2j*np.pi*t_n_matrix/T)

    c = np.reshape(c, (len(c),1))

    z_n = np.matmul(exp_matrix,c)
    z_n = np.reshape(z_n, (len(t),))
    
    return z_n

z_n = compute_z_n(c,T,dx,N)

z_n_func = lambda x: z_n[int(x/dx)]

## 3. Plot The Results

Try increasing the value of N to get better approximations
$$ \lim_{N \to \infty} z_{N} (t) = \boldsymbol{z} (t)

In [38]:
t = np.linspace(0,T,int(T/dx+1))
z = z_func(t)

def plot_fourier(N=N):
    c = fourier_coefficients(z_func,T,N)
    z_n = compute_z_n(c,T,dx,N)
    fig = plt.figure()
    ax = fig.add_axes([0.1,0.1,0.9,0.9])

    ax.plot(np.real(z),np.imag(z),label = 'z')
    ax.plot(np.real(z_n),np.imag(z_n),label = 'z_n')

    ax.legend(loc=0)

In [42]:
ipywidgets.interact(plot_fourier, N = (0,50,1)) #number of spinning vectors = 2N+1

interactive(children=(IntSlider(value=10, description='N', max=50), Output()), _dom_classes=('widget-interact'…

<function __main__.plot_fourier(N=10)>

In [40]:
radius = np.sqrt(np.real(c)**2+np.imag(c)**2)
def get_angle(c):
    return np.imag(np.log(c))
argument = get_angle(c)

In [43]:
%%manim -ql FourierSeries


#Animation
class FourierSeries(Scene):
    def get_vector_order(self):
        order = [0]
        s = 1
        for i in range(2*N):
            order.append(s*(1+i//2))
            s = -s
        return order

    def compute_vector_path(self, prev_path,ts, n):
        return prev_path + c[n+N]*np.exp(2j*np.pi*n*ts/T)

    def get_center_funcs(self):
        def functionize(path):
                return lambda x: path[int(x/dx)]
        
        ts = np.linspace(0,T,int(T/dx+1))
        prev_path = [0 for _ in range(len(ts))]
        center_funcs = []
        order = self.get_vector_order()

        for n in order:
            center_func = functionize(prev_path)
            center_funcs.append(center_func)
            prev_path = self.compute_vector_path(prev_path, ts, n)
        return center_funcs
    
    def get_vectors_and_circles(self):
        center_funcs = self.get_center_funcs()
        order = self.get_vector_order()
        vectors = VGroup()
        circles = VGroup()

        for i, n in enumerate(order):
            center_func = center_funcs[i]
            
            vector = self.get_vector(n,center_func)
            circle = self.get_circle(i,n,center_func)
            vectors.add(vector)
            circles.add(circle)
        return vectors, circles

    def vector_updater(self,vector,n,center_func):
        time = self.time.get_value()
        phase = argument[n+N]
        rad = radius[n+N]
        start_pos = vector.get_start()
        end_pos = [np.real(center_func(time)),np.imag(center_func(time)),0]

        vector.set_angle(phase+time*2*np.pi*n/T)
        vector.set_length(rad)
        vector.shift(end_pos-start_pos)

        return vector
    
    def get_vector(self, n, center_func):
        phase = argument[n+N]
        rad = radius[n+N]
        beg_pos = [np.real(center_func(0)),np.imag(center_func(0)),0]

        vector = Vector(RIGHT, buff = 0, fill_opacity = 0.75)
        vector.set_length(rad)
        vector.tip.scale(min(1.5,rad))
        vector.set_angle(phase)
        vector.shift(beg_pos)
        vector.add_updater(lambda vector, n=n, center_func=center_func: self.vector_updater(vector,n,center_func))
        
        return vector
    
    def get_circle(self, i, n, center_func):
        rad = radius[n+N]
        beg_pos = [np.real(center_func(0)),np.imag(center_func(0)),0]

        circle = Circle(radius = rad, color = BLUE_D, stroke_width = 2,stroke_opacity = 0.5).shift(beg_pos).shift(2*RIGHT)
        circle.add_updater(lambda circle, center_func=center_func: self.circle_updater(circle,center_func))
        return circle

    def circle_updater(self,circle,center_func):
        time = self.time.get_value()
        start_pos = circle.get_center()
        end_pos = [np.real(center_func(time)),np.imag(center_func(time)),0]

        circle.shift(end_pos-start_pos)
        return circle

    def construct(self):
        self.time = ValueTracker(0)
        vectors, circles = self.get_vectors_and_circles()

        z_parametric = always_redraw(
            lambda: ParametricFunction(lambda t: (np.real(z_n_func(t)), np.imag(z_n_func(t)), 0), t_range=[0,self.time.get_value()], color = YELLOW)
            )
        
        self.add(vectors, circles, z_parametric)
        self.play(self.time.animate.set_value(T),run_time = 6, rate_func = linear)

        




            
        